In [2]:
import math
import numpy as np
import keras
from keras import ops
from keras import layers
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import pandas as pd
from glob import glob

SEED = 42
keras.utils.set_random_seed(SEED)

In [3]:
#Parámetros

BASE_PATH = 'C:/Users/Richard/Desktop/Entorno Tesis/'
PATH = os.path.join(BASE_PATH, 'data/DogFaceNet_224resized/after_4_bis/')       # Ruta a imágenes
PATH_SAVE = os.path.join(BASE_PATH, 'output/history/')
PATH_MODEL = os.path.join(BASE_PATH, 'output/model/')

INPUT_SHAPE = (224, 224, 3)                           # Tamaño de entrada

# Data
BUFFER_SIZE = 512
BATCH_SIZE = 256

# Augmentation
IMAGE_SIZE = 72
PATCH_SIZE = 6
NUM_PATCHES = (IMAGE_SIZE // PATCH_SIZE) ** 2

# Optimizador
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001

# Épocas de entrenamiento
EPOCHS = 50

# Arquitectura
LAYER_NORM_EPS = 1e-6
TRANSFORMER_LAYERS = 8
PROJECTION_DIM = 64
NUM_HEADS = 4
TRANSFORMER_UNITS = [
    PROJECTION_DIM * 2,
    PROJECTION_DIM,
]
MLP_HEAD_UNITS = [2048, 1024]


In [6]:
#Carga del dataset

from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_LIST = os.path.join(BASE_PATH, 'data/DogFaceNet_224resized/classes_train.txt')
TEST_LIST = os.path.join(BASE_PATH, 'data/DogFaceNet_224resized/classes_test.txt')

# Leer listas de IDs
with open(TRAIN_LIST, "r") as f:
    train_ids = [line.strip() for line in f.readlines()]

with open(TEST_LIST, "r") as f:
    test_ids = [line.strip() for line in f.readlines()]

print("Num IDs train:", len(train_ids))
print("Num IDs test:", len(test_ids))

# Crear DataFrame de train
train_files = []
train_labels = []
for c in train_ids:
    files = glob(os.path.join(PATH, c, "*"))
    train_files.extend(files)
    train_labels.extend([c] * len(files))

train_df = pd.DataFrame({"filename": train_files, "class": train_labels})

# Crear DataFrame de test
test_files = []
test_labels = []
for c in test_ids:
    files = glob(os.path.join(PATH, c, "*"))
    test_files.extend(files)
    test_labels.extend([c] * len(files))

test_df = pd.DataFrame({"filename": test_files, "class": test_labels})

print("Train samples:", len(train_df))
print("Test samples:", len(test_df))

# Generadores
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="filename",
    y_col="class",
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical"
)

test_gen = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="filename",
    y_col="class",
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical"
)

Num IDs train: 7666
Num IDs test: 697
Train samples: 64594
Test samples: 3861
Found 64594 validated image filenames belonging to 1254 classes.
Found 3861 validated image filenames belonging to 139 classes.
